In [1]:
%pwd

'c:\\Arjun_Works\\NextStep-AI\\research'

In [2]:
import os

os.chdir("../")

%pwd

'c:\\Arjun_Works\\NextStep-AI'

In [5]:
from langchain_community.document_loaders import PyPDFLoader

In [6]:
document = PyPDFLoader("data/Computer_Science.pdf").load()

In [7]:
document[0].metadata

{'producer': 'Adobe Photoshop for Macintosh -- Image Conversion Plug-in',
 'creator': 'Adobe Photoshop CS5.1 Macintosh',
 'creationdate': '2015-03-11T00:07:47+05:30',
 'author': 'J. Glenn Brookshear',
 'moddate': '2016-12-05T22:16:28+08:00',
 'title': 'Computer Science: An Overview 12/E',
 'source': 'data/Computer_Science.pdf',
 'total_pages': 641,
 'page': 0,
 'page_label': 'Cover'}

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

In [9]:
chunks = text_splitter.split_documents(document)

In [10]:
chunks

[Document(metadata={'producer': 'Adobe Photoshop for Macintosh -- Image Conversion Plug-in', 'creator': 'Adobe Photoshop CS5.1 Macintosh', 'creationdate': '2015-03-11T00:07:47+05:30', 'author': 'J. Glenn Brookshear', 'moddate': '2016-12-05T22:16:28+08:00', 'title': 'Computer Science: An Overview 12/E', 'source': 'data/Computer_Science.pdf', 'total_pages': 641, 'page': 1, 'page_label': 'Title'}, page_content='An Overview\ncomputer science\nJ. Glenn Brookshear\nand\nDennis Brylow\n12th Edition\nGlobal Edition\nBoston   Columbus   Indianapolis   New York   San Francisco   Upper Saddle River\nAmsterdam   Cape Town   Dubai   London   Madrid   Milan   Munich   Paris   Montréal   Toronto\nDelhi   Mexico City   São Paulo   Sydney   Hong Kong   Seoul   Singapore   Taipei   Tokyo\nGlobal Edition contributions by\nManasa S.\nA01_BROO1160_12_SE_FM.indd   1 01/08/14   9:37 AM'),
 Document(metadata={'producer': 'Adobe Photoshop for Macintosh -- Image Conversion Plug-in', 'creator': 'Adobe Photoshop 

In [11]:
from dotenv import load_dotenv

load_dotenv()

True

In [13]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001"
)

In [16]:
%pip install faiss-cpu

from langchain.vectorstores import FAISS


vectorstore = FAISS.from_documents(chunks, embeddings)
vectorstore.save_local("knowledge_index")

Note: you may need to restart the kernel to use updated packages.



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\phoen\anaconda3\envs\llms\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\phoen\anaconda3\envs\llms\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\phoen\anaconda3\envs\llms\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\phoen\anaconda3\envs\llms\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  Fil

AttributeError: _ARRAY_API not found

ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).